In [1]:
%pip install fastapi pydantic toml openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import time
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    ts = time.time()

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

    duration = time.time() - ts
    
    print(f"delta time: {duration}")


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)



print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

context: Now, you are standing in front of the haunted mansion. 
The mansion is old and decrepit, with ivy growing up the walls and the windows boarded up. 
The front door is slightly ajar, inviting you inside...
actions:
1. Enter the mansion.
2. Take some photos of the mansion's exterior.
3. Look around the building.


delta time: 12.431968927383423
I select option 2. TURN: 2 


context: As you ascend the creaky staircase, you notice the hallway is blocked by debris from a fallen ceiling. The path to the upstairs rooms is obstructed, and it looks like it will require some effort to clear.
skill: strength
difficulty: hard

delta time: 5.611001968383789
I am making a skill check using STRENGTH against a difficulty of HARD.
And I rolled a 18 for a result of SUCCESS.
TURN: 3 


context: With considerable effort, you manage to clear the debris blocking the upstairs hallway. Your path is now clear, revealing rooms that beckon with secrets untold and perhaps dangers unseen.
summary: Harvey Walters, utilizing his strength, successfully cleared the debris blocking the upstairs hallway, opening the path to explore further.

delta time: 7.325313091278076
